# **Import Library**

In [0]:
import pandas as pd
import numpy as np

# **Import File**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_one = pd.read_csv('/content/drive/My Drive/work/nectec/dataset7.csv',parse_dates=['Date'])
with pd.ExcelFile('/content/drive/My Drive/work/nectec/dataSet4 (2)-newFromITaaS.xlsx') as reader:
  # print(reader.sheet_names)
  df_two = pd.read_excel(reader, sheet_name=reader.sheet_names[1])
df = pd.merge(df_one, df_two, on='Item No', how='inner')


# **table total receipt**

In [4]:
from calendar import monthrange
import plotly.graph_objects as go

def table_total_receipt(df,start_date,end_date):
  df['sum_price'] = df['Price'] * df['Quantity']
  df = df[['Receipt No','Item No','Price','Quantity','Date','Description','sum_price']]
  df['year'] = df['Date'].dt.year
  df['month'] = df['Date'].dt.month
  df['date'] = df['Date'].dt.day
  df['quarter'] = df['Date'].dt.quarter
  df['Date'] = df['year'].map(lambda x: str(x) + '-') + df['month'].map(lambda x: str(x) + '-') + df['month'].map(lambda x: str(x))
  df['Date'] = pd.to_datetime(df['Date'])
  df = df.groupby(['Receipt No','year','month','date','Date']).count().reset_index()[['Receipt No','year','month','date','Date','Item No']]
  df = df.rename(columns={"Item No": "count_receipt"})
  df_between_date = df[(df['Date'] > start_date) & (df['Date'] <= end_date)]
  df_between_date['quarter'] = df_between_date['Date'].dt.quarter
  df_between_date = df_between_date.reset_index()
  many_receipt = df_between_date[df_between_date['count_receipt'] > 1]
  many_receipt['category_receipt'] = 'many_sku'
  one_receipt = df_between_date[df_between_date['count_receipt'] == 1]
  one_receipt['category_receipt'] = 'one_sku'
  df_sum = pd.concat([many_receipt,one_receipt])
  df_sum = df_sum.sort_index()
  length_many_sku = df_sum[df_sum['category_receipt'] == 'many_sku']['count_receipt'].sum()
  length_one_sku = df_sum[df_sum['category_receipt'] == 'one_sku']['count_receipt'].sum()

  headerColor = 'mediumblue'
  rowEvenColor = 'white'
  rowOddColor = 'lightsteelblue'

  fig = go.Figure(data=[go.Table(
    header=dict(
      values=['<b>Date Range</b>','<b>Type</b>','<b>จำนวนใบเสร็จ</b>'],
      line_color='darkslategray',
      fill_color=headerColor,
      align=['left','center'],
      font=dict(color='white', size=12)
    ),
    cells=dict(
      values=[
        [f'{start_date}-{end_date}', f'{start_date}-{end_date}'],
        ['SKU เดียว', 'SKU ร่วม','SKU ทั้งหมด'],
        [f'{length_one_sku}', f'{length_many_sku}',f'{length_one_sku+length_many_sku}']],
      line_color='darkslategray',
      fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
      align = ['left', 'center'],
      font = dict(color = 'darkslategray', size = 11)
      ))
  ])
  return fig.show()
table_total_receipt(df,'2018-01-01','2019-12-18')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

# **Table Cumulative(not success)**

In [5]:
def clean_cumulative(df):
  df_all_columns = pd.merge(df_one, df_two, on='Item No', how='inner')
  df_all_columns['sum_price'] = df_all_columns['Price'] * df_all_columns['Quantity']
  df_clean_all = df_all_columns[['Receipt No','Item No','Price','Quantity','Date','Description','sum_price']]
  df_sum_item = df_clean_all.groupby(['Item No','Date']).sum().reset_index()
  df_sum_item = df_sum_item[['Receipt No','Item No','Date','sum_price','Quantity']]
  return df_sum_item
# df = clean_cumulative(df)

def choose_income_quality(choose_income_quality):
  # print(choose)
  df_choose_item = clean_cumulative(df).sort_values(f"{choose_income_quality}",ascending=False)
  df_choose_item[f'total_{choose_income_quality}'] = df_choose_item[f'{choose_income_quality}'].sum()
  df_choose_item[f'percent_{choose_income_quality}'] = df_choose_item[f'{choose_income_quality}']/df_choose_item[f'total_{choose_income_quality}'] * 100
  df_choose_item[f'cumulative_{choose_income_quality}'] = df_choose_item[f'percent_{choose_income_quality}'].cumsum()
  return df_choose_item
# df = choose_income_quality('sum_price')
# df = choose_income_quality('Quantity')

# def split_cumulative(percent):
#   return df[-int(len(df_choose_item) / 100 * percent):]

def show_comulative(category_choose,start_date, end_date,percent):
  df_between_date = choose_income_quality(f'{category_choose}').loc[(choose_income_quality(f'{category_choose}')['Date'] > start_date) & (choose_income_quality(f'{category_choose}')['Date'] <= end_date)]
  df_split_cumulative = df_between_date[df_between_date[f'cumulative_{category_choose}'] > percent]
  headerColor = 'mediumblue'
  rowEvenColor = 'white'
  rowOddColor = 'lightsteelblue'
  fig = go.Figure(data=[go.Table(
    header=dict(
      values=['<b>SKU</b>','<b>ยอดขายทั้งหมด</b>','<b>จำนวนสินค้าที่ขายได้</b>','<b>ยอดขายสะสม</b>'],
      line_color='darkslategray',
      fill_color=headerColor,
      align=['left','center'],
      font=dict(color='white', size=12)
    ),
    cells=dict(
      values=[
        df_split_cumulative['Receipt No'].tolist(),
        df_split_cumulative['sum_price'].tolist(),
        df_split_cumulative['Quantity'].tolist(),
        df_split_cumulative[f'cumulative_{category_choose}'].tolist()
        ],
      line_color='darkslategray',
      fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
      align = ['left', 'center'],
      font = dict(color = 'darkslategray', size = 11)
      ))
  ])
  return fig.show()
# show_comulative('sum_price','2018-01-01','2018-12-12',20)
show_comulative('Quantity','2018-01-01','2018-12-12',20)


Output hidden; open in https://colab.research.google.com to view.

# **Table Price**

In [6]:
def total_sell(category_sum,start_date,end_date):
  df_between_date = df[(df['Date'] > start_date) & (df['Date'] <= end_date)]
  if 'Quantity' == category_sum:
    name_column_count = 'จำนวนยอดขาย'
    name_column_category = 'หน่วย (รายการ)'
    category_sum_count = 'ชิ้น'
  elif 'Price' == category_sum:
    name_column_count = 'จำนวนรายได้'
    name_column_category = 'หน่วย (บาท)'
    category_sum_count = 'บาท'
  headerColor = 'mediumblue'
  rowEvenColor = 'white'
  rowOddColor = 'lightsteelblue'
  fig = go.Figure(data=[go.Table(
    header=dict(
      values=['<b>ช่วงเวลา</b>',f'<b>{name_column_count}</b>',f'<b>{name_column_category}</b>'],
      line_color='darkslategray',
      fill_color=headerColor,
      align=['left','center'],
      font=dict(color='white', size=12)
    ),
    cells=dict(
      values=[
        [f'{start_date}-{end_date}'],
        [df_between_date[f'{category_sum}'].sum()],
        [f'{category_sum_count}'],
        # df_split_cumulative['sum_price'].tolist(),
        # df_split_cumulative['Quantity'].tolist(),
        # df_split_cumulative[f'cumulative_{category_choose}'].tolist()
        ],
      line_color='darkslategray',
      fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
      align = ['left', 'center'],
      font = dict(color = 'darkslategray', size = 11)
      ))
  ])
  
  return fig.show()
# total_sell('Price','2018-01-01','2018-12-12')
total_sell('Quantity','2018-01-01','2018-12-12')
# show_comulative('sum_price','2018-01-01','2018-12-12',20)
# show_comulative('Quantity','2018-01-01','2018-12-12',20)

# **Tabel ตารางเเจ้งว่ามีสินค้าอะไรบ้าง**